1. 這份是用來爬取 Google Search 的搜尋列表結果以及各自的內文
2. 會需要預備好關鍵字清單，這邊用的是以「貝果」為主題在 Ubersuggest 爬回來的關鍵字們

# 1. 資料匯入－關鍵字清單

In [1]:
import pandas as pd

In [ ]:
file_path = "/content/drive/MyDrive/程式/bagel_flavor.csv"
selected_keyword = pd.read_csv(file_path).drop(columns = "Unnamed: 0")
selected_keyword = selected_keyword.rename(columns = {"Id":"keyword"})
selected_keyword

# 2. 資料爬取
---

In [ ]:
# 套件下載
!pip install readability-lxml
!pip install requests

## 2.1 爬蟲函示

In [4]:
import requests
import time
from bs4 import BeautifulSoup
from urllib.parse import unquote
from readability import Document
import requests.packages.urllib3
requests.packages.urllib3.disable_warnings()

def crawler_v2(keyword_list):
    try:
        stamp = 0
        post_list = []
        for keyword in keyword_list:
            stamp+=1
            print(stamp, ":",keyword)

            for page in range(0, 50, 10):
                    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
                    url = f'https://www.google.com/search?q={keyword}&start={page}'
                    res = requests.get(url,verify=False,headers=headers)
                    res.encoding = res.apparent_encoding
                    soup = BeautifulSoup(res.text,"lxml")
                    post = soup.find_all('div',class_='Gx5Zad fP1Qef xpd EtOod pkphOe')
                    # link = unquote(post[i].find('a')['href'].split('url=')[1].split('&')[0])

                    [post_list.append(
                            {"Title":post[i].find('a').find('div', class_='BNeawe vvjwJb AP7Wnd UwRFLe').text, 
                             "Description":post[i].find('div', class_='BNeawe s3v9rd AP7Wnd').text,
                             "Link": unquote(post[i].find('a')['href'].split('url=')[1].split('&')[0]),
                             "Content": "",
                            "Page":str(i+1)+"/"+str(page+10)[0] ,
                            "Source":keyword}) for i in range(len(post))]
                    time.sleep(2)

        posts = pd.DataFrame(post_list)
        
    except Exception as e:
        print(e)
        posts = pd.DataFrame(post_list)
        return posts 
    
    return posts

def content_crawler_v2(url):
    try:        
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
        res = requests.get(url,verify=False,headers=headers)
        if res.encoding == "ISO-8859-1":
            res.encoding = "utf-8"
        elif (res.encoding.lower() != "utf-8")&(res.encoding.lower() != "windows-1254"):
            res.encoding = res.apparent_encoding  
        doc = Document(res.text)
        content = BeautifulSoup(doc.summary(),"lxml").text
        if content == '\n':
            content = pixnet_to_be_detectd(res)
    except AttributeError:
        content = "AttributeError"
    except:
        content = "Error"
    return content

def pixnet_to_be_detectd(res):    
    soup = BeautifulSoup(res.text,"lxml")
    post = soup.find_all('div',class_='article-content-inner')[0]
    p_tags = post.find_all('p')
    p_text = []
    for tag in p_tags:
        if tag.text !="": p_text.append(tag.text)
    content = "，".join(p_text)
    return content

## 2.2 資料收集（1/2）－ Google Search 頁面爬取

### 利用 crawler_v2 函示爬取

In [ ]:
keyword_list = list(selected_keyword['keyword'])
keyword_list

['藍莓貝果',
 '起司貝果',
 '藍莓貝果食譜',
 '切達起司貝果',
 '乳酪貝果',
 'bagel貝果',
 '芝麻貝果',
 '起司火腿貝果',
 '貝果乳酪醬',
 '草莓貝果',
 '藍莓乳酪貝果',
 '起司q貝果',
 '貝果價格']

In [ ]:
# keyword_search_reault = pd.read_csv("keyword_search_reault(1-14).csv").drop(columns = 'Unnamed: 0')
keyword_search_reault = crawler_v2(keyword_list)
keyword_search_reault

1 : 藍莓貝果
2 : 起司貝果
3 : 藍莓貝果食譜
4 : 切達起司貝果
5 : 乳酪貝果
6 : bagel貝果
7 : 芝麻貝果
8 : 起司火腿貝果
9 : 貝果乳酪醬
10 : 草莓貝果
11 : 藍莓乳酪貝果
12 : 起司q貝果
13 : 貝果價格


,Title,Description,Link,Content,Page,Source
0,è—�è�“è²�æ�œby Sidneyçš„å»šæˆ¿æ¨‚åœ’ - æ„›æ–™ç�†,2017å¹´10æœˆ31æ—¥ Â· é«˜ç­‹éºµç²‰. 300å…‹ Â· é...,https://icook.tw/recipes/226733,,1/1,藍莓貝果
1,è—�è�“è²�æ�œby cookingç‘ªç‘ª - æ„›æ–™ç�†,1 å°�æ™‚ 30 åˆ†é�˜2022å¹´3æœˆ12æ—¥ Â· è‡ªå·±çƒ...,https://icook.tw/recipes/405724,,2/1,藍莓貝果
2,è„†çš®è—�è�“è²�æ�œBlueberry bagel - YouTube,2021å¹´6æœˆ13æ—¥ Â· æ��æ–™é…�åˆ†Ingredientsé«˜...,https://www.youtube.com/watch?v=PuyKNFsrIoc,,3/1,藍莓貝果
3,å’¬å¾—åˆ°æ–°é®®è—�è�“çš„è—�è�“è²�æ�œ - Robisto...,2015å¹´1æœˆ18æ—¥ Â· æ²’æ™‚é–“çš„æ™‚å€™ï¼Œæœ€å–...,https://robi-kitchenlab.blogspot.com/2015/01/b...,,4/1,藍莓貝果
4,ç´�ç´„é¢¨è—�è�“è²�æ�œ| NY Style Blueberry Bage...,2018å¹´3æœˆ29æ—¥ Â· é€™ä»½è—�è�“è²�æ�œå±¬æ–¼ç´...,https://brianrecipe.com/%E7%B4%90%E7%B4%84%E9%...,,5/1,藍莓貝果
...,...,...,...,...,...,...
631,麥當勞期間限定早餐【現烤焙果】原味貝果Pauly奶油乳酪抹醬,冰心仙子提供貝果價位和上市時間給大家參考 不想錯過美味貝果就要好好把握時間哦!!錯過不知道...,https://iceheart888.pixnet.net/blog/post/43758...,,6/5,貝果價格
632,鬆軟綿密芋頭風味芋頭雜糧貝果Taro Multigrain Bagel $60 - 馬可先生,手作雜糧貝果. 芋頭雜糧貝果. Taro Multigrain Bagel. 價格: $60...,https://www.mrmark.com.tw/index.php/product/vi...,,7/5,貝果價格
633,台北信義美食:好丘貝果聯名星級貝果堡，享受南村有光音樂光雕,2022年8月31日 · 價位雖偏高，但這煙燻鮭魚豪邁地放了許多層以上，難怪吃了許多，都還能...,https://banbi.tw/goodchos-bagel/,,8/5,貝果價格
634,台中市集美食｜手作有溫度的台中貝果專賣店，超過25種口味,2021年12月3日 · 治癒貝果菜單上多達26種口味，價格超平價，最貴的包餡貝果一顆50元...,https://juniorbro.tw/heal-bagel/,,9/5,貝果價格


In [ ]:
# pd.set_option('display.max_rows', 600)
# keyword_search_reault.head(100)
# keyword_search_reault.to_csv("/content/drive/MyDrive/程式/Crawler/flavor_search_reault.csv")

### 資料分割
1. target_posts -> 目標data
2. excluded_posts -> Link包含 'youtube','instagram','products','taobao','shopee','pchome' 的data

In [7]:
# keyword_search_reault = pd.read_csv("/content/drive/MyDrive/程式/Crawler/flavor_search_reault.csv").drop(columns = 'Unnamed: 0')
# keyword_search_reault

,Title,Description,Link,Content,Page,Source
0,è—�è�“è²�æ�œby Sidneyçš„å»šæˆ¿æ¨‚åœ’ - æ„›æ–™ç�†,2017å¹´10æœˆ31æ—¥ Â· é«˜ç­‹éºµç²‰. 300å…‹ Â· é...,https://icook.tw/recipes/226733,NaN,1/1,藍莓貝果
1,è—�è�“è²�æ�œby cookingç‘ªç‘ª - æ„›æ–™ç�†,1 å°�æ™‚ 30 åˆ†é�˜2022å¹´3æœˆ12æ—¥ Â· è‡ªå·±çƒ...,https://icook.tw/recipes/405724,NaN,2/1,藍莓貝果
2,è„†çš®è—�è�“è²�æ�œBlueberry bagel - YouTube,2021å¹´6æœˆ13æ—¥ Â· æ��æ–™é…�åˆ†Ingredientsé«˜...,https://www.youtube.com/watch?v=PuyKNFsrIoc,NaN,3/1,藍莓貝果
3,å’¬å¾—åˆ°æ–°é®®è—�è�“çš„è—�è�“è²�æ�œ - Robisto...,2015å¹´1æœˆ18æ—¥ Â· æ²’æ™‚é–“çš„æ™‚å€™ï¼Œæœ€å–...,https://robi-kitchenlab.blogspot.com/2015/01/b...,NaN,4/1,藍莓貝果
4,ç´�ç´„é¢¨è—�è�“è²�æ�œ| NY Style Blueberry Bage...,2018å¹´3æœˆ29æ—¥ Â· é€™ä»½è—�è�“è²�æ�œå±¬æ–¼ç´...,https://brianrecipe.com/%E7%B4%90%E7%B4%84%E9%...,NaN,5/1,藍莓貝果
...,...,...,...,...,...,...
631,麥當勞期間限定早餐【現烤焙果】原味貝果Pauly奶油乳酪抹醬,冰心仙子提供貝果價位和上市時間給大家參考 不想錯過美味貝果就要好好把握時間哦!!錯過不知道...,https://iceheart888.pixnet.net/blog/post/43758...,NaN,6/5,貝果價格
632,鬆軟綿密芋頭風味芋頭雜糧貝果Taro Multigrain Bagel $60 - 馬可先生,手作雜糧貝果. 芋頭雜糧貝果. Taro Multigrain Bagel. 價格: $60...,https://www.mrmark.com.tw/index.php/product/vi...,NaN,7/5,貝果價格
633,台北信義美食:好丘貝果聯名星級貝果堡，享受南村有光音樂光雕,2022年8月31日 · 價位雖偏高，但這煙燻鮭魚豪邁地放了許多層以上，難怪吃了許多，都還能...,https://banbi.tw/goodchos-bagel/,NaN,8/5,貝果價格
634,台中市集美食｜手作有溫度的台中貝果專賣店，超過25種口味,2021年12月3日 · 治癒貝果菜單上多達26種口味，價格超平價，最貴的包餡貝果一顆50元...,https://juniorbro.tw/heal-bagel/,NaN,9/5,貝果價格


In [8]:
target_posts = keyword_search_reault[~keyword_search_reault["Link"].str.contains("youtube|products|facebook|instagram")].copy()
target_posts

,Title,Description,Link,Content,Page,Source
0,è—�è�“è²�æ�œby Sidneyçš„å»šæˆ¿æ¨‚åœ’ - æ„›æ–™ç�†,2017å¹´10æœˆ31æ—¥ Â· é«˜ç­‹éºµç²‰. 300å…‹ Â· é...,https://icook.tw/recipes/226733,NaN,1/1,藍莓貝果
1,è—�è�“è²�æ�œby cookingç‘ªç‘ª - æ„›æ–™ç�†,1 å°�æ™‚ 30 åˆ†é�˜2022å¹´3æœˆ12æ—¥ Â· è‡ªå·±çƒ...,https://icook.tw/recipes/405724,NaN,2/1,藍莓貝果
3,å’¬å¾—åˆ°æ–°é®®è—�è�“çš„è—�è�“è²�æ�œ - Robisto...,2015å¹´1æœˆ18æ—¥ Â· æ²’æ™‚é–“çš„æ™‚å€™ï¼Œæœ€å–...,https://robi-kitchenlab.blogspot.com/2015/01/b...,NaN,4/1,藍莓貝果
4,ç´�ç´„é¢¨è—�è�“è²�æ�œ| NY Style Blueberry Bage...,2018å¹´3æœˆ29æ—¥ Â· é€™ä»½è—�è�“è²�æ�œå±¬æ–¼ç´...,https://brianrecipe.com/%E7%B4%90%E7%B4%84%E9%...,NaN,5/1,藍莓貝果
5,è—�è�“ä¹³é…ªè²�æ�œ è²�æ�œå…‰æ»‘å°�ç§˜è¨£ - ä¿...,2019å¹´5æœˆ20æ—¥ Â· é€™æ¬¡å…§é¤¡æ”¾äº†å¥¶æ²¹ä¹...,https://yummymum.tw/blueberrybagel/,NaN,6/1,藍莓貝果
...,...,...,...,...,...,...
631,麥當勞期間限定早餐【現烤焙果】原味貝果Pauly奶油乳酪抹醬,冰心仙子提供貝果價位和上市時間給大家參考 不想錯過美味貝果就要好好把握時間哦!!錯過不知道...,https://iceheart888.pixnet.net/blog/post/43758...,NaN,6/5,貝果價格
632,鬆軟綿密芋頭風味芋頭雜糧貝果Taro Multigrain Bagel $60 - 馬可先生,手作雜糧貝果. 芋頭雜糧貝果. Taro Multigrain Bagel. 價格: $60...,https://www.mrmark.com.tw/index.php/product/vi...,NaN,7/5,貝果價格
633,台北信義美食:好丘貝果聯名星級貝果堡，享受南村有光音樂光雕,2022年8月31日 · 價位雖偏高，但這煙燻鮭魚豪邁地放了許多層以上，難怪吃了許多，都還能...,https://banbi.tw/goodchos-bagel/,NaN,8/5,貝果價格
634,台中市集美食｜手作有溫度的台中貝果專賣店，超過25種口味,2021年12月3日 · 治癒貝果菜單上多達26種口味，價格超平價，最貴的包餡貝果一顆50元...,https://juniorbro.tw/heal-bagel/,NaN,9/5,貝果價格


In [9]:
excluded_posts = keyword_search_reault[keyword_search_reault["Link"].str.contains("youtube|products|facebook|instagram")].copy()
excluded_posts

,Title,Description,Link,Content,Page,Source
2,è„†çš®è—�è�“è²�æ�œBlueberry bagel - YouTube,2021å¹´6æœˆ13æ—¥ Â· æ��æ–™é…�åˆ†Ingredientsé«˜...,https://www.youtube.com/watch?v=PuyKNFsrIoc,NaN,3/1,藍莓貝果
9,è„†çš®è—�è�“è²�æ�œä»¥å‰�æˆ‘é‚„åœ¨ä¸Šç�­çš„æ™‚å...,2021å¹´6æœˆ13æ—¥ Â· è„†çš®è—�è�“è²�æ�œä»¥å‰�æˆ...,https://www.facebook.com/LaMaShania/videos/%E8...,NaN,10/1,藍莓貝果
15,藍莓起士貝果(奶素) 人氣第一 - Miopane,$70.00 供應中內容物：自製新鮮藍莓果醬、Cream Cheese 月銷萬顆的藍莓起士貝...,https://www.miopane.com/products/blueberry-bag...,NaN,6/2,藍莓貝果
16,石臼全麥藍莓貝果(全素) - Miopane,$70.00 供應中石臼全麥藍莓貝果(全素). 使用法國進口的天然全麥麵粉與酸甜的野生藍莓乾...,https://www.miopane.com/products/whole-wheat-b...,NaN,7/2,藍莓貝果
18,藍莓貝果 - 聖瑪莉,2022年6月20日 · 使用新鮮藍莓加入貝果. ... 商品介紹. 使用新鮮藍莓加入貝果喜...,https://www.sunmerry.com.tw/zh-TW/products/401...,NaN,9/2,藍莓貝果
...,...,...,...,...,...,...
563,重乳酪貝果(奶素) - Miopane,$75.00 供應中內容物：高達起士、巴馬起士Q彈的貝果麵團中有著滿滿的高達起士大口咬下每一...,https://www.miopane.com/products/double-cheese...,NaN,7/3,起司q貝果
594,吳寶春-原味貝果5入,"$200.00 供應中獨家研發的麵粉黃金比例發酵出的麵糰不但更香貝果的風味也更加濃厚以少糖,...",https://www.wupaochun.com/products/%E5%8E%9F%E...,NaN,8/1,貝果價格
615,藍莓果粒貝果- 麵包 - 星巴克,藍莓果粒貝果. Blueberry Bagel. 60. #植物基產品將經典貝果加入豐富的藍...,https://www.starbucks.com.tw/products/food/pro...,NaN,10/3,貝果價格
622,軟歐藏心貝果 - 路易莎,藏心貝果 ; 紫薯知心. Yam Paste · (奶素) ; 麥香知心黃金流沙. Whea...,https://www.louisacoffee.co/products_list/14,NaN,7/4,貝果價格


In [ ]:
# topic = target_posts[target_posts['Source'].str.contains("貝果推薦|貝果網購|貝果專賣店|貝果早午餐")].copy()
# topic.reset_index(drop=True, inplace=True)
# topic

## 2.3 資料收集（2/2）－ 文章頁面內文爬取

In [10]:
search_reault = target_posts.iloc[:100].copy().reset_index(drop=True) 
search_reault['Content'] = search_reault['Link'].apply(lambda link: content_crawler_v2(link))
search_reault

ERROR:readability.readability:error getting summary: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/readability/readability.py", line 213, in summary
    self._html(True)
  File "/usr/local/lib/python3.8/dist-packages/readability/readability.py", line 148, in _html
    self.html = self._parse(self.input)
  File "/usr/local/lib/python3.8/dist-packages/readability/readability.py", line 157, in _parse
    doc, self.encoding = build_doc(input)
  File "/usr/local/lib/python3.8/dist-packages/readability/htmls.py", line 21, in build_doc
    doc = lxml.html.document_fromstring(
  File "/usr/local/lib/python3.8/dist-packages/lxml/html/__init__.py", line 761, in document_fromstring
    raise etree.ParserError(
lxml.etree.ParserError: Document is empty


,Title,Description,Link,Content,Page,Source
0,è—�è�“è²�æ�œby Sidneyçš„å»šæˆ¿æ¨‚åœ’ - æ„›æ–™ç�†,2017å¹´10æœˆ31æ—¥ Â· é«˜ç­‹éºµç²‰. 300å…‹ Â· é...,https://icook.tw/recipes/226733,\nSidney的廚房樂園\n\n354 \n食譜\n\n\n1.2 萬\n粉絲\n\n我是...,1/1,藍莓貝果
1,è—�è�“è²�æ�œby cookingç‘ªç‘ª - æ„›æ–™ç�†,1 å°�æ™‚ 30 åˆ†é�˜2022å¹´3æœˆ12æ—¥ Â· è‡ªå·±çƒ...,https://icook.tw/recipes/405724,"\ncooking瑪瑪\n\n102 \n食譜\n\n\n1,223 \n粉絲\n\n從事平...",2/1,藍莓貝果
2,å’¬å¾—åˆ°æ–°é®®è—�è�“çš„è—�è�“è²�æ�œ - Robisto...,2015å¹´1æœˆ18æ—¥ Â· æ²’æ™‚é–“çš„æ™‚å€™ï¼Œæœ€å–...,https://robi-kitchenlab.blogspot.com/2015/01/b...,\n\n\n\n\n\n沒時間的時候，最喜歡做貝果當隔天的早餐，\n\n低油低糖、省時快速又...,4/1,藍莓貝果
3,ç´�ç´„é¢¨è—�è�“è²�æ�œ| NY Style Blueberry Bage...,2018å¹´3æœˆ29æ—¥ Â· é€™ä»½è—�è�“è²�æ�œå±¬æ–¼ç´...,https://brianrecipe.com/%E7%B4%90%E7%B4%84%E9%...,\n\n藍莓貝果是屬於紐約風格做法 (另有經典蒙特婁貝果做法、口感更具嚼勁)。而人生第一次吃...,5/1,藍莓貝果
4,è—�è�“ä¹³é…ªè²�æ�œ è²�æ�œå…‰æ»‘å°�ç§˜è¨£ - ä¿...,2019å¹´5æœˆ20æ—¥ Â· é€™æ¬¡å…§é¤¡æ”¾äº†å¥¶æ²¹ä¹...,https://yummymum.tw/blueberrybagel/,\n\n又是一道要消耗食材的作品~\n想到家裡冷凍庫還有冷凍的藍莓，冷藏室還有一點奶油乳酪~...,6/1,藍莓貝果
...,...,...,...,...,...,...
95,新手烘焙》貝果食譜作法~簡單揉一揉就可以啦！ - Yuki's Life,"評分 5.0 (1) DIY, 原味, 手作, 新手烘焙, 方法, 材料, 簡單, 簡單...",https://yukiblog.tw/read-3677.html,\n\n\n\n\n對於要揉的麵包我實在是有點害怕，因為聽會做麵包的媽媽說，光是揉到麵團出筋...,2/3,藍莓貝果食譜
96,[烘焙食譜] 4款我最愛的貝果食譜。藍莓貝果、芝麻貝果、伯爵茶貝果,2021年5月23日 · 食譜大公開! 夏天大家最愛吃貝果麵包，因為貝果擁有低脂、低膽固醇、...,https://www.sisiwander.com/2021/05/4.html,,3/3,藍莓貝果食譜
97,藍莓貝果(Blueberry Bagels) @ liwen幸福家政 - 隨意窩,藍莓貝果(Blueberry Bagels)~日式貝果成品數量:18個(每個100g) 製作...,https://blog.xuite.net/magic151510/twblog/3189...,浪漫柔美的粉紫色、風味十足的藍莓貝果。\n \n\n\n\n\n藍莓貝果(Blueberry...,4/3,藍莓貝果食譜
98,藍莓貝果 - U Blog,2021年11月7日 · 藍莓貝果食譜（6個）. ﻿. 材料：. 高筋粉250G、全麥粉50...,https://blog.ulifestyle.com.hk/article/chengsf...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n成功發佈\n影片...,5/3,藍莓貝果食譜


In [ ]:
# search_reault_rest = target_posts.iloc[100:].copy().reset_index(drop=True)
# search_reault_rest['Content'] = search_reault_rest['Link'].apply(lambda link: content_crawler_v2(link))
# search_reault_rest

# pd.set_option('display.max_rows', 600)
# search_reault_rest

In [ ]:
# search_reault_rest.to_csv("search_reault_rest.csv")

# 3. 資料清理

## 3.1 清理函示
1. clean_data -> 將文本中的特殊字符、空格等等刪除
2. str_almost_no_chinese -> 將非中文文本資料排除

In [ ]:
test = search_reault.dropna()
test

100

##  3.2 資料清理（1/4）－ 特殊字符移除

In [11]:
# 將文本中的特殊字符、空格等等刪除
import re
def clean_data(content):
    try:
        a = re.compile(r'\n|&nbsp|\xa0|\\xa0|\u3000|\\u3000|\\u0020|\u0020|\t|\r') 
        content = a.sub('', content)
#         print(content)
    except:
        index = target_index(search_reault,'Content',content)
        print(content)
        print(index,':{無法移除特殊字符}')
    return content

def target_index(data, col, value):
    return data[data[col]==str(value)].index

In [12]:
search_reault["Content"] = search_reault["Content"].apply(lambda sen: clean_data(sen))
search_reault

,Title,Description,Link,Content,Page,Source
0,è—�è�“è²�æ�œby Sidneyçš„å»šæˆ¿æ¨‚åœ’ - æ„›æ–™ç�†,2017å¹´10æœˆ31æ—¥ Â· é«˜ç­‹éºµç²‰. 300å…‹ Â· é...,https://icook.tw/recipes/226733,"Sidney的廚房樂園354食譜1.2萬粉絲我是Sidney,婚後離開台灣旅居過美東,美西,...",1/1,藍莓貝果
1,è—�è�“è²�æ�œby cookingç‘ªç‘ª - æ„›æ–™ç�†,1 å°�æ™‚ 30 åˆ†é�˜2022å¹´3æœˆ12æ—¥ Â· è‡ªå·±çƒ...,https://icook.tw/recipes/405724,"cooking瑪瑪102食譜1,223粉絲從事平面設計10年，婚後轉行做料理、居家佈置部落客...",2/1,藍莓貝果
2,å’¬å¾—åˆ°æ–°é®®è—�è�“çš„è—�è�“è²�æ�œ - Robisto...,2015å¹´1æœˆ18æ—¥ Â· æ²’æ™‚é–“çš„æ™‚å€™ï¼Œæœ€å–...,https://robi-kitchenlab.blogspot.com/2015/01/b...,沒時間的時候，最喜歡做貝果當隔天的早餐，低油低糖、省時快速又超好吃，新鮮藍莓的酸甜感，讓貝果...,4/1,藍莓貝果
3,ç´�ç´„é¢¨è—�è�“è²�æ�œ| NY Style Blueberry Bage...,2018å¹´3æœˆ29æ—¥ Â· é€™ä»½è—�è�“è²�æ�œå±¬æ–¼ç´...,https://brianrecipe.com/%E7%B4%90%E7%B4%84%E9%...,藍莓貝果是屬於紐約風格做法(另有經典蒙特婁貝果做法、口感更具嚼勁)。而人生第一次吃到貝果已經...,5/1,藍莓貝果
4,è—�è�“ä¹³é…ªè²�æ�œ è²�æ�œå…‰æ»‘å°�ç§˜è¨£ - ä¿...,2019å¹´5æœˆ20æ—¥ Â· é€™æ¬¡å…§é¤¡æ”¾äº†å¥¶æ²¹ä¹...,https://yummymum.tw/blueberrybagel/,又是一道要消耗食材的作品~想到家裡冷凍庫還有冷凍的藍莓，冷藏室還有一點奶油乳酪~秤重一下，恩...,6/1,藍莓貝果
...,...,...,...,...,...,...
95,新手烘焙》貝果食譜作法~簡單揉一揉就可以啦！ - Yuki's Life,"評分 5.0 (1) DIY, 原味, 手作, 新手烘焙, 方法, 材料, 簡單, 簡單...",https://yukiblog.tw/read-3677.html,對於要揉的麵包我實在是有點害怕，因為聽會做麵包的媽媽說，光是揉到麵團出筋有薄膜就花了她好幾個...,2/3,藍莓貝果食譜
96,[烘焙食譜] 4款我最愛的貝果食譜。藍莓貝果、芝麻貝果、伯爵茶貝果,2021年5月23日 · 食譜大公開! 夏天大家最愛吃貝果麵包，因為貝果擁有低脂、低膽固醇、...,https://www.sisiwander.com/2021/05/4.html,,3/3,藍莓貝果食譜
97,藍莓貝果(Blueberry Bagels) @ liwen幸福家政 - 隨意窩,藍莓貝果(Blueberry Bagels)~日式貝果成品數量:18個(每個100g) 製作...,https://blog.xuite.net/magic151510/twblog/3189...,浪漫柔美的粉紫色、風味十足的藍莓貝果。藍莓貝果(BlueberryBagels)~日式貝果成...,4/3,藍莓貝果食譜
98,藍莓貝果 - U Blog,2021年11月7日 · 藍莓貝果食譜（6個）. ﻿. 材料：. 高筋粉250G、全麥粉50...,https://blog.ulifestyle.com.hk/article/chengsf...,"成功發佈影片已經成功發佈,您現在可以隨時查閱、編輯或刪改您的影片。文章已經成功發佈,您現在可...",5/3,藍莓貝果食譜


##  3.3 資料清理（2/4）－ 非中文文本移除

In [13]:
def str_almost_no_chinese(str): # True為非中文、False為中文
    try:
        if not str:
            print(str)
            pass
        ch_str = 0
        not_ch_str = 0
        for ch in str:
            if u'\u4e00' <= ch <= u'\u9fa5':
                ch_str+=1
            else:
                not_ch_str+=1
        if ch_str <= not_ch_str:
            print(str)
            print("-----------------------------------")
            return True
    except:
        print(str)
    return False

In [14]:
no_chinese = search_reault[search_reault['Content'].apply(lambda sen: str_almost_no_chinese(sen))]
no_chinese



-----------------------------------
豆穌朋食品股份有限公司|客服專線：02-29109475總公司:新北市新店區寶興路45巷9弄8號2樓service@dosuper.com.twCopyright©DoSuperFoodsCo.,Ltd.AllRightsReserved請尊重智慧財產權著作權為豆穌朋食品股份有限公司所有轉載必究
-----------------------------------


-----------------------------------
Error
-----------------------------------
Error
-----------------------------------
Error
-----------------------------------
Error
-----------------------------------
Error
-----------------------------------
Error
-----------------------------------
Error
-----------------------------------
Error
-----------------------------------
Error
-----------------------------------
Error
-----------------------------------
Error
-----------------------------------
Error
-----------------------------------
Error
-----------------------------------
Error
-----------------------------------


-----------------------------------
Error
-----------------------------------
Error
-----------------------------------
Error
-----------------------

,Title,Description,Link,Content,Page,Source
6,è—�è�“è²�æ�œ- å„ªæƒ æ�¨è–¦- 2022å¹´12æœˆ| è�¦ç...,ã€�å�Ÿå‘³æ™‚ä»£ã€‘ç�¾è²¨æ¸›é†£æ¸›ç³–è²�æ�œå�£å...,https://shopee.tw/search?keyword=%E8%97%8D%E8%...,,8/1,藍莓貝果
13,貝果-藍莓乳酪5入 - 豆穌朋,【貝果-藍莓乳酪5入】(得記). 迎合台灣人口味，量身訂做的改良式貝果; 每一口都咬得到果粒...,https://www.dosuper.com.tw/product/$$$_a0ca134...,豆穌朋食品股份有限公司|客服專線：02-29109475總公司:新北市新店區寶興路45巷9弄...,8/2,藍莓貝果
14,日式貝果｜善菓屋,綜合貝果6入. NT$ 255. 加入購物車. 乳酪貝果. NT$ 48. 加入購物車. 藍...,https://www.gooddeedstw.com/pages/snshopbagel,,1/3,藍莓貝果
15,【RICH S】美國進口正統美式貝果113gx6個x4袋(原味貝果/藍莓貝果,Rich's Plain Bagel原味貝果/Rich's Blueberry Bagel藍...,https://www.momoshop.com.tw/goods/GoodsDetail....,Error,4/3,藍莓貝果
16,野生藍莓貝果 - 好好貝果,野生藍莓貝果. 特選加拿大冷凍野生藍莓，. 急速冷凍技術保留藍莓的營養價值，. 健康又衛生～...,https://www.hohobagel.com.tw/product/76aee1af-...,Error,5/3,藍莓貝果
17,藍莓貝果| 莎士比亞烘焙坊- 與巴黎、東京同步的在地幸福,$45.00 供應中藍莓貝果. 特選野生小藍莓粒加入彈Q的麵糰中一同攪打，嚼勁十足，有濃厚的...,https://www.shakespeare2010.com/product/produc...,Error,6/3,藍莓貝果
18,"紐約貝果: 原味、芝麻、藍莓、草莓Bagel (Original, Sesame ...",2019年10月19日 · 紐約貝果: 原味、芝麻、藍莓、草莓Bagel (Original...,https://lauraslilkitchen.com/%E7%B4%90%E7%B4%8...,Error,7/3,藍莓貝果
19,【哈肯舖】藍莓貝果(5入/袋) - ihergo愛合購,商品說明. 【藍莓貝果】選用被譽為Superfood的藍莓乾，豐富的維他命C含量、微酸甜的滋...,https://www.ihergo.com/mobile/store/fast-hogan...,Error,8/3,藍莓貝果
20,【貝果】可可奶油藍莓 - 布朗主廚貝果專賣Golden Brown,$58.00【貝果】可可奶油藍莓. 售價：$58 / 個全館消費滿$2500 (含) 以上，...,https://www.goldenbrown.com.tw/goldenbrown/ind...,Error,9/3,藍莓貝果
21,藍莓貝果- 手感麵包 - 哈肯舖,$58.00 供應中被譽為Superfood的藍莓乾，含有豐富的抗氧化物及花青素，搭配低脂、...,https://www.hoganbakery.com.tw/product_d.php?l...,Error,10/3,藍莓貝果


In [15]:
search_reault.drop(index = no_chinese.index, inplace = True)
search_reault.reset_index(inplace=True,drop=True)
search_reault

,Title,Description,Link,Content,Page,Source
0,è—�è�“è²�æ�œby Sidneyçš„å»šæˆ¿æ¨‚åœ’ - æ„›æ–™ç�†,2017å¹´10æœˆ31æ—¥ Â· é«˜ç­‹éºµç²‰. 300å…‹ Â· é...,https://icook.tw/recipes/226733,"Sidney的廚房樂園354食譜1.2萬粉絲我是Sidney,婚後離開台灣旅居過美東,美西,...",1/1,藍莓貝果
1,è—�è�“è²�æ�œby cookingç‘ªç‘ª - æ„›æ–™ç�†,1 å°�æ™‚ 30 åˆ†é�˜2022å¹´3æœˆ12æ—¥ Â· è‡ªå·±çƒ...,https://icook.tw/recipes/405724,"cooking瑪瑪102食譜1,223粉絲從事平面設計10年，婚後轉行做料理、居家佈置部落客...",2/1,藍莓貝果
2,å’¬å¾—åˆ°æ–°é®®è—�è�“çš„è—�è�“è²�æ�œ - Robisto...,2015å¹´1æœˆ18æ—¥ Â· æ²’æ™‚é–“çš„æ™‚å€™ï¼Œæœ€å–...,https://robi-kitchenlab.blogspot.com/2015/01/b...,沒時間的時候，最喜歡做貝果當隔天的早餐，低油低糖、省時快速又超好吃，新鮮藍莓的酸甜感，讓貝果...,4/1,藍莓貝果
3,ç´�ç´„é¢¨è—�è�“è²�æ�œ| NY Style Blueberry Bage...,2018å¹´3æœˆ29æ—¥ Â· é€™ä»½è—�è�“è²�æ�œå±¬æ–¼ç´...,https://brianrecipe.com/%E7%B4%90%E7%B4%84%E9%...,藍莓貝果是屬於紐約風格做法(另有經典蒙特婁貝果做法、口感更具嚼勁)。而人生第一次吃到貝果已經...,5/1,藍莓貝果
4,è—�è�“ä¹³é…ªè²�æ�œ è²�æ�œå…‰æ»‘å°�ç§˜è¨£ - ä¿...,2019å¹´5æœˆ20æ—¥ Â· é€™æ¬¡å…§é¤¡æ”¾äº†å¥¶æ²¹ä¹...,https://yummymum.tw/blueberrybagel/,又是一道要消耗食材的作品~想到家裡冷凍庫還有冷凍的藍莓，冷藏室還有一點奶油乳酪~秤重一下，恩...,6/1,藍莓貝果
...,...,...,...,...,...,...
58,新鮮藍莓貝果食譜、做法 - 烘焙筆記Baking Note,步驟說明 · 1 材料準備. 先將藍莓跟牛奶倒一起，用果汁機打碎，再加入鋼盆一起攪拌。 · ...,https://bakingnote.com/recipe/404/magazine,用1000ml滾水+80g砂糖，沸騰後轉小火，貝果雙面各燙20秒，前輩有教我烘培紙要剪開比較...,8/2,藍莓貝果食譜
59,鮮芒果貝果、藍莓貝果、九層塔貝果食譜與作法分享| 喵媽水波爐食譜,2021年3月31日 · 貝果的作法有很多種，每個人對貝果的口感喜好度都不一樣。有些人偏好有...,https://kimiyo.tw/bagels/,鮮芒果貝果、藍莓貝果、九層塔貝果食譜與作法分享|喵媽水波爐食譜LastUpdatedon20...,1/3,藍莓貝果食譜
60,新手烘焙》貝果食譜作法~簡單揉一揉就可以啦！ - Yuki's Life,"評分 5.0 (1) DIY, 原味, 手作, 新手烘焙, 方法, 材料, 簡單, 簡單...",https://yukiblog.tw/read-3677.html,對於要揉的麵包我實在是有點害怕，因為聽會做麵包的媽媽說，光是揉到麵團出筋有薄膜就花了她好幾個...,2/3,藍莓貝果食譜
61,藍莓貝果(Blueberry Bagels) @ liwen幸福家政 - 隨意窩,藍莓貝果(Blueberry Bagels)~日式貝果成品數量:18個(每個100g) 製作...,https://blog.xuite.net/magic151510/twblog/3189...,浪漫柔美的粉紫色、風味十足的藍莓貝果。藍莓貝果(BlueberryBagels)~日式貝果成...,4/3,藍莓貝果食譜


##  3.4 資料清理（3/4）－ 空值移除

In [16]:
search_reault.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63 entries, 0 to 62
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Title        63 non-null     object
 1   Description  63 non-null     object
 2   Link         63 non-null     object
 3   Content      63 non-null     object
 4   Page         63 non-null     object
 5   Source       63 non-null     object
dtypes: object(6)
memory usage: 3.1+ KB


##  3.5 資料清理（4/4）－ 異常移除

In [17]:
cookpad = search_reault[search_reault['Link'].str.contains("cookpad")]
cookpad

,Title,Description,Link,Content,Page,Source
9,【藍莓貝果】料理－ 14 篇食譜與家常做法- Cookpad,藍莓貝果料理怎麼做才好吃？這裡有14 篇超簡單家常食譜通通免費看！包括：新手藍莓貝果 、藍...,https://cookpad.com/tw/%E6%90%9C%E5%B0%8B/%E8%...,請稍等…系統偵測到你可能在使用自動工具瀏覽網站請驗證你不是機器人RefID:#64b4bfb...,3/2,藍莓貝果
18,【起司貝果】料理－ 68 篇食譜與家常做法- Cookpad,起司貝果料理怎麼做才好吃？這裡有68 篇超簡單家常食譜通通免費看！包括：乳酪番茄炒蛋貝果、南...,https://cookpad.com/tw/%E6%90%9C%E5%B0%8B/%E8%...,請稍等…系統偵測到你可能在使用自動工具瀏覽網站請驗證你不是機器人RefID:#fb23b8c...,4/1,起司貝果
51,ã€�è—�è�“è²�æ�œã€‘æ–™ç�†ï¼� 14 ç¯‡é£Ÿè­œèˆ‡å®¶...,è—�è�“è²�æ�œæ–™ç�†æ€�éº¼å�šæ‰�å¥½å�ƒï¼Ÿé€™è£¡æ...,https://cookpad.com/tw/%E6%90%9C%E5%B0%8B/%E8%...,請稍等…系統偵測到你可能在使用自動工具瀏覽網站請驗證你不是機器人RefID:#29da878...,9/1,藍莓貝果食譜
54,Fresh Blueberry Bagels-新鮮藍莓貝果 !!! 食譜與作法by Gra...,"如照片, 可依各人喜好喜歡有果粒口感的,用擀麵棍將此藍莓盆搗碎. 我個人喜愛有藍莓果粒口感的...",https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/2045...,請稍等…系統偵測到你可能在使用自動工具瀏覽網站請驗證你不是機器人RefID:#2b77bc4...,3/2,藍莓貝果食譜


In [18]:
search_reault.drop(index=cookpad.index, inplace=True)
search_reault.reset_index(inplace=True, drop=True)
search_reault

,Title,Description,Link,Content,Page,Source
0,è—�è�“è²�æ�œby Sidneyçš„å»šæˆ¿æ¨‚åœ’ - æ„›æ–™ç�†,2017å¹´10æœˆ31æ—¥ Â· é«˜ç­‹éºµç²‰. 300å…‹ Â· é...,https://icook.tw/recipes/226733,"Sidney的廚房樂園354食譜1.2萬粉絲我是Sidney,婚後離開台灣旅居過美東,美西,...",1/1,藍莓貝果
1,è—�è�“è²�æ�œby cookingç‘ªç‘ª - æ„›æ–™ç�†,1 å°�æ™‚ 30 åˆ†é�˜2022å¹´3æœˆ12æ—¥ Â· è‡ªå·±çƒ...,https://icook.tw/recipes/405724,"cooking瑪瑪102食譜1,223粉絲從事平面設計10年，婚後轉行做料理、居家佈置部落客...",2/1,藍莓貝果
2,å’¬å¾—åˆ°æ–°é®®è—�è�“çš„è—�è�“è²�æ�œ - Robisto...,2015å¹´1æœˆ18æ—¥ Â· æ²’æ™‚é–“çš„æ™‚å€™ï¼Œæœ€å–...,https://robi-kitchenlab.blogspot.com/2015/01/b...,沒時間的時候，最喜歡做貝果當隔天的早餐，低油低糖、省時快速又超好吃，新鮮藍莓的酸甜感，讓貝果...,4/1,藍莓貝果
3,ç´�ç´„é¢¨è—�è�“è²�æ�œ| NY Style Blueberry Bage...,2018å¹´3æœˆ29æ—¥ Â· é€™ä»½è—�è�“è²�æ�œå±¬æ–¼ç´...,https://brianrecipe.com/%E7%B4%90%E7%B4%84%E9%...,藍莓貝果是屬於紐約風格做法(另有經典蒙特婁貝果做法、口感更具嚼勁)。而人生第一次吃到貝果已經...,5/1,藍莓貝果
4,è—�è�“ä¹³é…ªè²�æ�œ è²�æ�œå…‰æ»‘å°�ç§˜è¨£ - ä¿...,2019å¹´5æœˆ20æ—¥ Â· é€™æ¬¡å…§é¤¡æ”¾äº†å¥¶æ²¹ä¹...,https://yummymum.tw/blueberrybagel/,又是一道要消耗食材的作品~想到家裡冷凍庫還有冷凍的藍莓，冷藏室還有一點奶油乳酪~秤重一下，恩...,6/1,藍莓貝果
5,è—�è�“è²�æ�œ FOOD - JUJUEMMA,2022å¹´3æœˆ12æ—¥ Â· å°�æ–¼æ¯�å¤©æ—©ä¸Šè¦�æƒ³æ—...,https://jujuemma.com/2022/03/12/%E8%97%8D-%E8%...,【♥️自製西式麵點】對於每天早上要想早餐內容常常滿苦惱，因為不喜歡重複性太高、也怕麻煩，8成...,7/1,藍莓貝果
6,ã€�7-11ã€‘Amelie's Bagelï¼šåˆ‡é�”èµ·å�¸è²�æ�œã...,2022å¹´4æœˆ11æ—¥ Â· ... çš„ç†±æ°´ç‡™ç…®ï¼Œæ‰“é...,https://www.smileyhuan.com/2022/04/amelies-bag...,統一集團Amelie'sBagel好久沒有更新超商麵包系列了，這次要來分享的是我最愛的麵包之...,9/1,藍莓貝果
7,藍莓貝果| 手揉Q彈咬勁、光滑小秘訣一次分享 - 食譜自由配,2021年2月17日 · 藍莓貝果| 手揉Q彈咬勁、光滑小秘訣一次分享 · 新鮮酵母粉先用開...,https://food.ltn.com.tw/article/958,食材（4人份）高筋麵粉250g新鮮酵母粉3g藍莓果乾20g調味料砂糖30g鹽1g西式早餐中的...,1/2,藍莓貝果
8,藍莓雙色貝果 - Carol 自在生活,2010年8月20日 · 藍莓雙色貝果 · 1.將新鮮藍莓果粒+冷水+檸檬汁使用果汁機打成泥...,https://caroleasylife.blogspot.com/2010/08/blo...,"朋友臨時有事,來電取消了中午的約會.我們一家三口就臨時起意到淡水逛老街.意外的決定更有驚喜,...",2/2,藍莓貝果
9,「影音」脆皮藍莓Bagel - 辣媽Shania,2021年6月13日 · 4.放到烤盤上，最後發酵50分鐘。 5.先預熱烤箱210度。 6....,https://www.lamashania.com/2021/06/bagel.html,高筋麵粉300g藍莓80g冰水90g砂糖10g酵母2g鹽巴3gVIDEO1.所有麵團材料放入...,4/2,藍莓貝果


In [21]:
# search_reault.to_csv("/content/drive/MyDrive/程式/flavor_search_reault_with_content.csv")